In [1]:
import os
from tqdm import tqdm
from utils import load_jsonl_iteratively
from collections import Counter

data_root = "/data/xzhao/dataset/meta_med/clean/"
seen_docs = set()
allowed_subjects = ["医学", "薬学", "歯学"]
failed_indices = []
en_keywords = Counter()
en_docs = []
for filename in os.listdir(data_root):
    if not filename.endswith(".jsonl"):
        continue
    bilingual_cnt = 0
    ja_cnt = 0
    en_cnt = 0
    for idx, item in tqdm(enumerate(load_jsonl_iteratively(os.path.join(data_root, filename)))):
        record = False
        doc_id = item['articleId']['journalCode'] + "@@" + item['articleId']['articleCode']
        if doc_id in seen_docs:
            continue
        seen_docs.add(doc_id)
        if 'subjects' in item['journalInfo'] and item['journalInfo']['subjects'] is not None:
            en_subjects = [subject['content'] for subject in item['journalInfo']['subjects'] if subject['language']=="en"]
            ja_subjects = [subject['content'] for subject in item['journalInfo']['subjects'] if subject['language']=="ja"]
            for sub in en_subjects:
                en_keywords[sub] += 1
                
            for subject in ja_subjects:
                record = any([asub in subject for asub in allowed_subjects])
                if record:
                    break
        if not record:
            continue
        
        langs = [_item['language'] for _item in item['abstract']]
        if len(langs) == 2 and "ja" in langs and "en" in langs:
            bilingual_cnt += 1
        if "ja" in langs:
            ja_cnt += 1
        if "en" in langs:
            en_cnt += 1
            en_docs.append(doc_id)
        else:
            failed_indices.append((idx, bilingual_cnt, doc_id))
    print(f"{filename}: {bilingual_cnt} code-switching abstracts found.")
    print(f"{filename}: {ja_cnt} Japanese-only abstracts found.")
    print(f"{filename}: {en_cnt} English-only abstracts found.")
    
print("All deduplicated documents with bilingual expression: ", len(seen_docs))

1083830it [00:22, 47369.93it/s]

metadata_med_ddp.jsonl: 404366 code-switching abstracts found.
metadata_med_ddp.jsonl: 614444 Japanese-only abstracts found.
metadata_med_ddp.jsonl: 404366 English-only abstracts found.
All deduplicated documents with bilingual expression:  1083830


In [ ]:
en_docs = set(en_docs)
backup_root = "/data/xzhao/dataset/roman-pretrain/instructions/ja/backup"

to_write = []
for item in tqdm(load_jsonl_iteratively(os.path.join(backup_root, "medical_trans.jsonl"))):
    if item['docid'] not in en_docs:
        continue
    to_write.append(item)
len(to_write)

1228878it [04:56, 4141.67it/s]


In [11]:
len(to_write)

808722

In [12]:
from utils import dump_jsonl

new_root = "/data/xzhao/dataset/roman-pretrain/instructions/ja/"
dump_jsonl(to_write, os.path.join(new_root, "medical_trans.jsonl"))

In [56]:
item['docid']

'cjpt@@2004/0/2004_0_E0157'

In [40]:
failed_indices[0], failed_indices[-1], len(failed_indices)

((652874, 404366, 'jceeek@@2012/0/2012_89'),
 (1083829, 404366, 'cjpt@@2004/0/2004_0_E0157'),
 210078)

In [37]:
min(zip(failed_indices)[0]), max(zip(*failed_indices)[0]), len(failed_indices)

TypeError: 'zip' object is not subscriptable

In [31]:
404366 + 210079

614445

In [29]:
item

{'articleId': {'journalCode': 'cjpt',
  'articleCode': '2004/0/2004_0_E0157',
  'articleCode_l0_deprecated': '2004_0_E0157'},
 'publicationDate': {'year': 2005},
 'title': [{'language': 'ja', 'content': '自治会型デイホームにおける転倒予防事業の取り組み（第2報）'}],
 'journalInfo': {'subjects': [{'language': 'ja', 'content': '一般医学・社会医学・看護学'},
   {'language': 'ja', 'content': '学際科学'},
   {'language': 'ja', 'content': '臨床医学'}]},
 'keywords': [{'language': 'ja', 'content': '自治会型デイホーム'}],
 'abstract': [{'language': 'ja',
   'content': '【はじめに】福井市において平成12年から高齢者の予防的福祉事業として、自治会型デイホーム事業を行っている。その事業の一環として平成15年度より、転倒予防ための講義及び福井市版の転倒予防体操を考案した。今回その転倒予防体操の効果について、年齢層及び転倒リスクの観点から比較検討したので報告する。 【転倒予防体操】福井市版転倒予防体操の特徴は、下肢筋力、立位バランスの改善を音楽にあわせて行われることである。頻度は各地区毎に週1回から月1回で、尚自宅での体操の実施を指導した。 【対象及び方法】対象者は、2ヶ月間の転倒予防体操を継続的に実施できた者で、その前後に身体機能評価（10m最大歩行速度(m/分)、40cm踏み台昇降、片脚立位時間、継ぎ足歩行、Functional Reach、Timed up & Go、握力）と転倒リスクチェック表（0から15点）の評価が可能であった67名とした。平均年齢は76.8歳で、性別はすべて女性であった。統計処理はt検定を用いた。 【結果】1、すべての対象者においては、体操実施前後で身体機能評価が有意に改善したものは認められなかった。2、75歳以

In [5]:
item

{'articleId': {'journalCode': 'sccj1979',
  'articleCode': '42/2/42_2_121',
  'articleCode_l0_deprecated': '42_2_121'},
 'publicationDate': {'year': 2008, 'month': 6, 'dayOfTheMonth': 20},
 'title': [{'language': 'en',
   'content': 'Makeup Inhibits Reduction of Reactive Oxygen Scavenging Enzyme Activity Induced by Mental Stress'},
  {'language': 'ja', 'content': 'メイクアップは精神的ストレスによる活性酸素消去酵素の活性低下を抑制する'}],
 'journalInfo': {'subjects': [{'language': 'ja', 'content': '薬学'},
   {'language': 'ja', 'content': 'ナノ・材料科学'},
   {'language': 'ja', 'content': '化学'},
   {'language': 'ja', 'content': '生物学・生命科学・基礎医学'},
   {'language': 'ja', 'content': '学際科学'}]},
 'keywords': [{'language': 'en', 'content': 'makeup'}],
 'abstract': [{'language': 'en',
   'content': 'Recent studies showed that makeup reduces mental stress, indicated by not only psychological but also physiological values. In this study, we examined the relationship between mental stress and the activity of reactive oxygen scavenging enzym